# Project: Long Beach Animal Shelter Intakes and Outcomes

## Description

**Objective**: \
\
Answer questions for various shareholders in the city of Long Beach, CA concerning intakes and outcomes at the local animal shelter.

**Dataset**: \
\
This dataset was pulled from the [Long Beach Open Data Portal](https://data.longbeach.gov/explore/dataset/animal-shelter-intakes-and-outcomes/). \
It is a 7.8MB CSV file containing intake and outcome data for animals captured by or surrendered to the city.

**Tools Used**:

- pandas
- Matplotlib
- Seaborn

# Introduction

For any city that has at least one animal shelter, there are various shareholders interested in how that shelter is run and what happens to the animals that pass through the shelter's doors.\
\
This analysis looks to answer questions for the following parties in Long Beach, CA:
- **Shelter managers:**
  - How long do animals typically stay in the shelter by species or intake condition?
  - What intake reasons are most strongly correlated with negative outcomes (e.g., euthanasia)?
  - Are there seasonal trends in animal intakes or outcomes?
- **Animal welfare advocates:**
  - What percentage of animals are adopted vs. euthanized, and how does that vary by type, sex, or condition?
  - Are there disparities in outcomes for specific breeds or geographic areas?
  - How many animals are returned to owners vs. adopted?
- **Local government officials:**
  - Is there a correlation between specific neighborhoods and high intake rates?
  - Has the shelter’s performance improved over time (e.g., reduced euthanasia rates)?
  - What’s the annual intake/output volume and trend?
- **Local citizenry:**
  - When is the best time of year to adopt (e.g., more animals available)?
  - What types of animals are most commonly available for adoption?
  - Can geographic patterns inform community outreach for fostering or adoption?
- **Internal analysts:**
  - What features best predict positive outcomes using logistic regression or clustering?
  - Can intake condition be used to forecast outcome types?

# Data handling

## Preview

In [1]:
# Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Import helper functions and variables
from utilities.config import get_path_obj, raw_data_path, processed_data_path, products_dir, images_dir

In [2]:
# Load data
df = pd.read_csv(raw_data_path)

# Preview
df.head()

,Animal ID,Animal Name,Animal Type,Primary Color,Secondary Color,Sex,DOB,Intake Date,Intake Condition,Intake Type,...,Outcome Type,Outcome Subtype,latitude,longitude,intake_is_dead,outcome_is_dead,was_outcome_alive,geopoint,intake_duration,is_current_month
0,A594350,*HEAVY CREAM,CAT,BLACK,NaN,Neutered,2014-07-28,2017-07-28,NORMAL,STRAY,...,ADOPTION,REPEAT ADT,33.799760,-118.126388,Alive on Intake,False,1,"33.7997598, -118.1263884",81.0,0
1,A347815,DUKE,DOG,BLACK,TAN,Neutered,2005-04-14,2018-11-30,NORMAL,OWNER SURRENDER,...,RESCUE,LIVELOVE,33.799760,-118.126388,Alive on Intake,False,1,"33.7997598, -118.1263884",27.0,0
2,A707449,*TABITHA,DOG,BLACK,WHITE,Spayed,2022-10-23,2023-09-23,NORMAL,STRAY,...,ADOPTION,NaN,33.798953,-118.167334,Alive on Intake,False,1,"33.7989532, -118.167334",18.0,0
3,A712850,*KIWI,DOG,BLONDE,GOLD,Spayed,2022-07-06,2024-02-03,NORMAL,RETURN,...,ADOPTION,WEB,33.798936,-118.195889,Alive on Intake,False,1,"33.7989357, -118.1958891",0.0,0
4,A738972,KITTEN 2,CAT,BLACK,NaN,Unknown,2025-03-28,2025-04-04,NORMAL,STRAY,...,RESCUE,LITTLELION,33.798936,-118.195889,Alive on Intake,False,1,"33.7989357, -118.1958891",0.0,0


In [3]:
# Structure and summary
df.info()
df.describe(include='all')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33707 entries, 0 to 33706
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Animal ID          33707 non-null  object 
 1   Animal Name        19956 non-null  object 
 2   Animal Type        33707 non-null  object 
 3   Primary Color      33707 non-null  object 
 4   Secondary Color    15964 non-null  object 
 5   Sex                33707 non-null  object 
 6   DOB                29433 non-null  object 
 7   Intake Date        33707 non-null  object 
 8   Intake Condition   33707 non-null  object 
 9   Intake Type        33707 non-null  object 
 10  Intake Subtype     33265 non-null  object 
 11  Reason for Intake  2230 non-null   object 
 12  Outcome Date       33381 non-null  object 
 13  Crossing           33707 non-null  object 
 14  Jurisdiction       33706 non-null  object 
 15  Outcome Type       33374 non-null  object 
 16  Outcome Subtype    298

,Animal ID,Animal Name,Animal Type,Primary Color,Secondary Color,Sex,DOB,Intake Date,Intake Condition,Intake Type,...,Outcome Type,Outcome Subtype,latitude,longitude,intake_is_dead,outcome_is_dead,was_outcome_alive,geopoint,intake_duration,is_current_month
count,33707,19956,33707,33707,15964,33707,29433,33707,33707,33707,...,33374,29842,33707.000000,33707.000000,33707,33707,33707.000000,33707,33381.000000,33707.000000
unique,32557,9996,10,80,44,5,5977,3079,16,12,...,18,240,NaN,NaN,1,2,NaN,10154,NaN,NaN
top,A637086,*,CAT,BLACK,WHITE,Male,2018-04-10,2023-05-03,NORMAL,STRAY,...,RESCUE,SPCALA,NaN,NaN,Alive on Intake,False,NaN,"33.8096122, -118.0826161",NaN,NaN
freq,8,104,16083,8548,9380,7739,40,54,15297,23719,...,7842,4074,NaN,NaN,33707,26766,NaN,570,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,33.815444,-118.149526,NaN,NaN,0.794078,NaN,18.741949,0.012075
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.139548,0.530849,NaN,NaN,0.404379,NaN,47.219301,0.109221
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,19.297815,-122.695911,NaN,NaN,0.000000,NaN,0.000000,0.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,33.783990,-118.190865,NaN,NaN,1.000000,NaN,0.000000,0.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,33.806783,-118.173175,NaN,NaN,1.000000,NaN,5.000000,0.000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,33.851210,-118.128915,NaN,NaN,1.000000,NaN,16.000000,0.000000


## Cleaning and preparation

## Exploratory Data Analysis (EDA)

## Deeper analysis and modeling

# Analysis

## Insights and recommendations

### Insights

### Recommedations

## Summary

*This report can also be found [here](../products/report.md).*

## Appendix